In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
from scipy.signal import resample,welch, butter,filtfilt,freqs
from sklearn.preprocessing import StandardScaler
from scipy.interpolate import RectBivariateSpline

In [2]:
#Following Class was provided to me by my supervisors for generating simulated HD-sEMG signals. 
#Although I adapted some parts to include template shifting and lowpassing I did not create this code.
#Credit goes to Professor Dario Farina's research group, specifically Alex Clarke for writing the code
#and Silvia Muceli for aquiring the templates.

class generateMUAPtrain:
    #This class uses templates from Silvia's model to build emg from MUAP trains.
    #The convolution function used causes edge effects, so to avoid this the 
    #MUAP train simulated is longer than specified by sampleLen, and then
    #cut to the desired length. 
    
    
    def __init__(self,sampleTime,numMU, lowpass = False, shift = False):       #number of samples of length sampleLen
        self.sampleLen = int(sampleTime*2048) + 200    #length of sample (samp Freq 4026Hz)
        self.numMU = numMU                  #number of motor units in sim
        self.lambdas = np.random.randint(10,15,numMU)/2048 #firing rate of motor units
        #load simulated MUAP templates:
        templates = np.load('templates.npy')[1000:,:,:,:]
        locations = np.load('fibreLocs.npy')[1000:,:]
        distance = np.sqrt(locations[:,0]**2 + locations[:,1]**2)
        templates = templates[np.argsort(distance),:,:,:]
        templates = templates[1:(numMU+1),:,:,:]
        #templates = templates[MU:(numMU+MU),:,:,:]
        self.numSensors = np.shape(templates)[1]*np.shape(templates)[2]
        templates = np.transpose(templates,(0,2,1,3))
        
        if shift != False:
            templates_shift = np.zeros(templates.shape)
            #shift = 10
            x_coord = np.array(range(8))
            y_coord = np.array(range(24))
            for MU in range(templates.shape[0]):
                for t in range(templates.shape[3]):
                    inter = RectBivariateSpline(x_coord, y_coord, templates[MU,:,:,t])
                    x_fine = np.array(range(0,8*10))/10
                    y_fine = np.array(range(-10,24*10))/10
                    MU_int = inter(x_fine, y_fine)
                    for x in range(templates.shape[1]):
                        for y in range(templates.shape[2]):
                            templates_shift[MU,x,y,t] = MU_int[x*10,y*10+(10-shift)]
            templates = templates_shift
        reshapeTemplates = np.zeros([np.shape(templates)[0],self.numSensors,np.shape(templates)[3]])
        count = 0
        for i in range(np.shape(templates)[1]):
            for j in range(np.shape(templates)[2]):
                reshapeTemplates[:,count,:] = templates[:,i,j,:]
                count+=1
        if lowpass != False:
            b, a = butter(5, lowpass/1024)
            reshapeTemplates = filtfilt(b, a, reshapeTemplates, axis=2)  
        
        self.templates = reshapeTemplates
        self.tempMax = np.trapz(np.abs(self.templates),axis=1)
            
            
    #nextSequence finds the timestamps of MU activations (using a poisson distribution)
    def nextSequence(self,lam):
        timeStamps = np.zeros(1,dtype=int)
        while timeStamps[-1] <= self.sampleLen:
            nextSpike = int(random.expovariate(lam))
            while nextSpike < 70 or nextSpike > 500: nextSpike = int(random.expovariate(lam))
            timeStamps = np.append(timeStamps,timeStamps[-1] + nextSpike)
        return timeStamps[1:-1]    
    
    #spikeTrain converts the timestamps into a binary time series MUAP train
    def spikeTrain(self):
        spikes = np.zeros((self.numMU,self.sampleLen))
        for i,j in enumerate(self.lambdas):
            spikes[i,self.nextSequence(j)-1] = 1
        return spikes
            
    #genSpikes uses the spikeTrain function to build an array of 
    #MU activation trains of size and shape specified on initalisation
    def genSpikes(self):
        MUtrain = np.zeros([self.sampleLen,self.numMU])
        MUtrain = np.transpose(self.spikeTrain())
        self.MUtrain = MUtrain
        
    #addNoise calculates and adds gaussian noise of desired SNR
    def addNoise(self,EMGsamples,SNR):
        data = np.transpose(EMGsamples)
        NoisyEMGsamples = np.zeros(np.shape(EMGsamples))
        for i in range(np.shape(data)[0]):
            sigWelch = welch(data[i,:],2048)
            sigPower = np.sum(sigWelch[1]) * (sigWelch[0][1] - sigWelch[0][0])
            noisePowerTarget = sigPower/(10**((SNR)/10))
            noise = np.random.normal(0,np.sqrt(noisePowerTarget),np.shape(data)[1])
            noiseWelch = welch(noise,2048)
            noisePower = np.sum(noiseWelch[1]) * (noiseWelch[0][1] - noiseWelch[0][0])
            realSNR = 10*np.log10(sigPower/noisePower)
            if (realSNR - SNR) > 1: 
                print("SNR warning")
                print(realSNR)
            NoisyEMGsamples[:,i] = EMGsamples[:,i] + np.transpose(noise)
        return NoisyEMGsamples
        
    #initSim builds the EMG from the MU activation trains by convolving
    #the binary MU trains with the templates 
    def initSim(self):
        self.genSpikes() 
        EMGsamples = np.zeros([self.sampleLen,self.numSensors])
        for k in range(self.numSensors):
                emg = np.zeros([self.numMU,self.sampleLen])
                for j in range(self.numMU):
                    train = np.squeeze(self.MUtrain[:,j])
                    template = np.squeeze(self.templates[j,k,:])
                    template = resample(template,128)
                    emg[j,:] = np.convolve(train,template,'same')
                emg = np.sum(emg,axis=0,keepdims=True)
                EMGsamples[:,k] = np.squeeze(np.transpose(emg))
        self.MUtrain = self.MUtrain[100:-100,:]
        self.EMGsamples = EMGsamples[100:-100,:]

    def returnOutput(self): return self.MUtrain
    
    def returnInput(self,SNR): return self.addNoise(self.EMGsamples,SNR)
    
    def returnCleanInput(self): return self.EMGsamples

In [3]:
#Generate 20 second HD-sEMG data based on given parameters: 
#MUs to be in the signal, noise in dB, template lowpassing cutoff in Hz and template longitudian shift in mm
def generateData(MUs, noise, lowpass, shift):
    length = 20
    if isinstance(lowpass, str):
        lowpass = False
    
    test = generateMUAPtrain(length, MUs, lowpass, shift)
    sim = test.initSim()
    spikes = test.returnOutput().astype(int)
    EMG = test.returnInput(noise)
         
    return EMG, spikes

In [4]:
#Generate K 20 second HD-sEMG segments to be used for K-fold cross validation
def generateKfolds(MUs, noise, lowpass, shift, k):
    save_xdata = []
    save_ydata = []
    for i in range(k):
        x, y = generateData(MUs, noise, lowpass, shift)
        save_xdata.append(x)
        save_ydata.append(y)
    return save_xdata, save_ydata

In [ ]:
k = 5
file_name = 'noise_data'

for n in [20,15,10,5,0]:
    noise = n
    MUs = 20
    lowpass = 'full'
    shift = 0
    xdata, y_data = generateKfolds(MUs, noise, lowpass, shift, k)
    for j in range(k):
        np.save('{}/{}dB_fold{}_x'.format(file_name, n, j+1),xdata[j])
        np.save('{}/{}dB_fold{}_y'.format(file_name, n, j+1),y_data[j])

In [ ]:
k = 5
file_name = 'MUs_data'

for n in [10,20,30,40,50]:
    noise = 30
    MUs = n
    lowpass = 'full'
    shift = 0
    xdata, y_data = generateKfolds(MUs, noise, lowpass, shift, k)
    for j in range(k):
        np.save('{}/{}dB_fold{}_x'.format(file_name, n, j+1),xdata[j])
        np.save('{}/{}dB_fold{}_y'.format(file_name, n, j+1),y_data[j])

In [ ]:
k = 5
file_name = 'lowpass_data'

for n in [500,300,200,150,125]:
    noise = 30
    MUs = 20
    lowpass = n
    shift = 0
    xdata, y_data = generateKfolds(MUs, noise, lowpass, shift, k)
    for j in range(k):
        np.save('{}/{}dB_fold{}_x'.format(file_name, n, j+1),xdata[j])
        np.save('{}/{}dB_fold{}_y'.format(file_name, n, j+1),y_data[j])

In [ ]:
k = 5
file_name = 'shift_data'

for n in [0,2,4,6,8]:
    noise = 30
    MUs = 20
    lowpass = 'full'
    shift = n
    xdata, y_data = generateKfolds(MUs, noise, lowpass, shift, k)
    for j in range(k):
        np.save('{}/{}dB_fold{}_x'.format(file_name, n, j+1),xdata[j])
        np.save('{}/{}dB_fold{}_y'.format(file_name, n, j+1),y_data[j])